In [1]:

import numpy as np
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,Conv2D,MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator    
from keras.applications import MobileNetV2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

2025-10-07 13:45:09.155152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759844709.376382      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759844709.430984      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train='/kaggle/input/fer2013/train'
test='/kaggle/input/fer2013/test'

In [3]:
emotions=['angry', 'happy', 'sad', 'surprise', 'neutral']

In [4]:
import tensorflow as tf


size=(128,128)


train_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


train_data = train_gen.flow_from_directory(
    train,
    target_size=size,
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=emotions
)

# Validation data (20%)
val_data = train_gen.flow_from_directory(
    train,
    target_size=size,
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=emotions
)

# Test data (separate unseen dataset)
test_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

test_data = test_gen.flow_from_directory(
    test,
    target_size=size,
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    classes=emotions
)


Found 19341 images belonging to 5 classes.
Found 4835 images belonging to 5 classes.
Found 6043 images belonging to 5 classes.


In [5]:
base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128,3))


base.trainable = True

model = Sequential([
    base,
    
    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),  
    Dense(5, activation='softmax')
])

I0000 00:00:1759844752.159518      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [6]:
from keras.callbacks import ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 4, 4, 64)       │       737,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 4, 4, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 4, 4, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,087,109 (11.78 MB)

 Trainable params: 3,052,613 (11.64 MB)

 Non-trainable params: 34,496 (134.75 KB)

In [8]:

model.fit(
    train_data,
    epochs=40,
    validation_data=val_data,
    callbacks=[early_stopping, reduce_lr]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40


I0000 00:00:1759844795.204428      80 service.cc:148] XLA service 0x7f7e60003c40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759844795.205252      80 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1759844798.758582      80 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1759844804.323340      80 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1759844804.520264      80 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


  1/605 ━━━━━━━━━━━━━━━━━━━━ 10:00:58 60s/step - accuracy: 0.1875 - loss: 1.9943

I0000 00:00:1759844816.444467      80 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


600/605 ━━━━━━━━━━━━━━━━━━━━ 1s 390ms/step - accuracy: 0.4302 - loss: 1.4467

E0000 00:00:1759845057.402332      81 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1759845057.598377      81 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


605/605 ━━━━━━━━━━━━━━━━━━━━ 387s 543ms/step - accuracy: 0.4310 - loss: 1.4452 - val_accuracy: 0.4360 - val_loss: 1.5593 - learning_rate: 0.0010
Epoch 2/40
605/605 ━━━━━━━━━━━━━━━━━━━━ 121s 201ms/step - accuracy: 0.6052 - loss: 1.1411 - val_accuracy: 0.4451 - val_loss: 1.5956 - learning_rate: 0.0010
Epoch 3/40
605/605 ━━━━━━━━━━━━━━━━━━━━ 120s 198ms/step - accuracy: 0.6429 - loss: 1.0825 - val_accuracy: 0.3837 - val_loss: 1.7829 - learning_rate: 0.0010
Epoch 4/40
605/605 ━━━━━━━━━━━━━━━━━━━━ 121s 201ms/step - accuracy: 0.6841 - loss: 1.0009 - val_accuracy: 0.6558 - val_loss: 1.0730 - learning_rate: 2.0000e-04
Epoch 5/40
605/605 ━━━━━━━━━━━━━━━━━━━━ 121s 200ms/step - accuracy: 0.7108 - loss: 0.9582 - val_accuracy: 0.6734 - val_loss: 1.0252 - learning_rate: 2.0000e-04
Epoch 6/40
605/605 ━━━━━━━━━━━━━━━━━━━━ 121s 200ms/step - accuracy: 0.7208 - loss: 0.9346 - val_accuracy: 0.6966 - val_loss: 0.9769 - learning_rate: 2.0000e-04
Epoch 7/40
605/605 ━━━━━━━━━━━━━━━━━━━━ 124s 205ms/step - accur

In [9]:
pred2=model.predict(test_data)

189/189 ━━━━━━━━━━━━━━━━━━━━ 63s 318ms/step


In [10]:

print(accuracy_score(test_data.labels, np.argmax(pred2, axis=1)))

0.7454906503392354


In [11]:
from sklearn.metrics import classification_report 
print(classification_report(test_data.classes,np.argmax(pred2, axis=1)))

              precision    recall  f1-score   support

           0       0.72      0.62      0.67       958
           1       0.88      0.89      0.88      1774
           2       0.63      0.64      0.63      1247
           3       0.87      0.83      0.85       831
           4       0.62      0.69      0.65      1233

    accuracy                           0.75      6043
   macro avg       0.74      0.73      0.74      6043
weighted avg       0.75      0.75      0.75      6043



In [12]:
model.save('mobilenetv2.keras')